In [63]:
_PROMPT_TEMPLATE = """
请你提取下面段落的关键词，要求所提取的关键词能够鲜明表示内容性质。所提取的关键词应不少于5个。

待处理内容：

本次股票发行后拟在创业板市场上市，该市场具有较高的投资风险。创业
板公司具有业绩不稳定、经营风险高、退市风险大等特点，投资者面临较大的
市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风险因
素，审慎作出投资决定。
华瑞电器股份有限公司
Huarui Electrical Appliance Co., Ltd.
册地址：宁波市鄞州区姜山镇科技园区
首次公开发行股票并在创业板上市
招股意向书
发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、
误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连
带的法律责任。
发行人及全体董事、监事、高级管理人员、发行人的控股股东、实际控制人
以及保荐人、承销的证券公司承诺因发行人招股意向书及其他信息披露资料有虚
假记载、误导性陈述或者重大遗漏，致使投资者在证券发行和交易中遭受损失的，
将依法赔偿投资者损失。

注意：你的回答不需要分段。你的回答格式应该按照下面的内容，请注意```output 等标记都必须输出，这是我用来提取答案的标记。
除了你的回答之外不要输出多余的文字。

```output
${{"keywords":[你提取的关键词]}}
"""

In [1]:
import dashscope
from dashscope import Generation
from http import HTTPStatus
import random
from CustomLLM import QWENonlie
apikey='sk-02b8b1a9175a4cae9bf9571d517d8b81'
llm=QWENonlie.from_name(model_name=Generation.Models.qwen_max,api_key=apikey)
#llm(_PROMPT_TEMPLATE)

In [6]:
%load_ext autoreload
%autoreload 2
from config import QWEN_CONFIG
from Container import g_container
from dashscope import TextEmbedding,BatchTextEmbedding
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import DashScopeEmbeddings,HuggingFaceEmbeddings
from TxtProcess import faiss_kb_ds
g_container.MODEL = llm
cfg = QWEN_CONFIG(api_key=apikey,
                  embedding_name=BatchTextEmbedding.Models.text_embedding_async_v1)
emb=DashScopeEmbeddings(model=TextEmbedding.Models.text_embedding_v1,
                        dashscope_api_key=apikey)
emb2=HuggingFaceEmbeddings(model_name='../../model/m3e/')
text_ld=TextLoader(file_path='./test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt')
sp=RecursiveCharacterTextSplitter(
        chunk_size=cfg.chunk_size, chunk_overlap=cfg.chunk_overlap)
docs=text_ld.load_and_split(text_splitter=sp)
fkb=faiss_kb_ds(vs_path=cfg.vec_store_path,kb_path='./test_data/',embs=emb2)
fkb.do_add_doc(docs[:2])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[{'id': 'c71c2fcf-ebf9-4fc8-a058-9b9ac8efc903',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': 'b8025a58-ad5b-4fc9-b320-108b9b43851d',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}}]

In [9]:
fkb.do_search("国元证券股份有限公司")

[(Document(page_content='保荐人承诺因其为发行人首次公开发行股票制作、出具的文件有虚假记载、误导性陈述或者重大遗漏，给投资者造成损失的，将先行赔偿投资者损失。发行股票类型：人民币普通股（A股）发行股数：公司拟首次公开发行股票总数不超过 2,167 万股，本次公开发行股份占发行后公司股份总数的比例不低于25%；本次公开发行全部为新股发行，老股东不公开发售股份。每股面值：人民币1.00元每股发行价格：[ ]元预计发行日期：2016年10月13日拟上市证券交易所：深圳证券交易所发行后总股本：不超过8,667万股本次发行前股东所持股份的流通限制及自愿锁定股份的承诺：1、公司控股股东、实际控制人之一、董事长余春明，公司股东、实际控制人之一、董事、总经理余超彪，胡建飞、汪红时、叶松林、刘松林、朱观润等 5名持有公司股份的董事、监事、高级管理人员承诺（其中，持有公司股份的监事仅承诺第（1）项与第（2）项）：（1）自股份公司股票在证券交易所上市交易之日起三十六个月内，不转让或者委托他人管理本次发行前其已持有的股份公司股份，也不由股份公司回购该部分股份。（2）在上述锁定期届满后，若其仍在股份公司任职的，在股份公司任职期间每年转让的股份公司股份不超过其所持有股权总数的25%；自其从股份公司处离职半年内，不转让所持有的股份公司股份；自其申报离任六个月后的十二个月内通过证券交易所挂牌交易出售的股份公司股票数量占其所持有股份公司股票总数的比例不超过50%。（3）所持公司股票在锁定期满后两年内减持的，减持价格不低于发行价；公司上市后 6 个月内如公司股票连续20个交易日的收盘价均低于发行价，或者上市后6个月期末收盘价低于发行价，本人所持公司股票的锁定期限自动延长 6 个月。上述减持价格和股份锁定承诺不因本人职务变更、离职而终止。（前述发行价指公司首次公开发行股票的发行价格，如果公司上市后因派发现金红利、送股、转增股本、增发新股等原因进行除权、除息的，则按照证券交易所的有关规定作除权除息处理。）2、公司股东余春禄、张光秀、陈发娣、余志平承诺：自股份公司股票在证券交易所上市交易之日起三十六个月内不转让或者委托他人管理本次发行前其已持有的股份公司股份，也不由股份公司回购该部分股份。3、除上述股东外，本次发行前的公司其他 67 名自然人股东承诺：自股份公司股票在证券交易所上市交

In [8]:
fkb.do_search(query='余春禄',top_k=5,threshold=1)

[]

In [7]:
ppppp="""
本次股票发行后拟在创业板市场上市，该市场具有较高的投资风险。创业
板公司具有业绩不稳定、经营风险高、退市风险大等特点，投资者面临较大的
市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风险因
素，审慎作出投资决定。
华瑞电器股份有限公司
Huarui Electrical Appliance Co., Ltd.
册地址：宁波市鄞州区姜山镇科技园区
首次公开发行股票并在创业板上市
招股意向书
发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、
误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连
带的法律责任。
发行人及全体董事、监事、高级管理人员、发行人的控股股东、实际控制人
以及保荐人、承销的证券公司承诺因发行人招股意向书及其他信息披露资料有虚
假记载、误导性陈述或者重大遗漏，致使投资者在证券发行和交易中遭受损失的，
将依法赔偿投资者损失。
"""
%load_ext autoreload
%autoreload 2
from Tools.keywords_extra import LLMKeywordsEXChain
testk=LLMKeywordsEXChain.from_llm(llm=llm)
testk._call({"inputs":ppppp})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['发行人',
 '风险',
 '意向书',
 '招股',
 '误导性',
 '遗漏',
 '监事',
 '投资者',
 '管理人员',
 '陈述',
 '创业板',
 '董事',
 '市场',
 '华瑞',
 '高级',
 '承诺',
 '损失',
 '记载',
 '后拟',
 '股票发行',
 '创业板市场',
 '投资风险',
 '创业公司',
 '业绩不稳定',
 '经营风险高',
 '退市风险大',
 '市场风险',
 '华瑞电器股份有限公司',
 'Huarui Electrical Appliance Co., Ltd.',
 '宁波市鄞州区姜山镇科技园区',
 '首次公开发行股票',
 '招股意向书',
 '虚假记载',
 '误导性陈述',
 '重大遗漏',
 '法律责任',
 '保荐人',
 '承销的证券公司',
 '信息披露资料',
 '赔偿投资者损失']

In [15]:
adsasdsd='\n请你提取下面段落的关键词，要求所提取的关键词能够鲜明表示内容性质。所提取的关键词应不少于20个。\n同时，对于内容中存在的公司、机构名称，地址信息，人名等，也作为关键词。\n\n待处理的内容：\n\n本次股票发行后拟在创业板市场上市，该市场具有较高的投资风险。创业\n板公司具有业绩不稳定、经营风险高、退市风险大等特点，投资者面临较大的\n市场风险。投资者应充分了解创业板市场的投资风险及本公司所披露的风险因\n素，审慎作出投资决定。\n华瑞电器股份有限公司\nHuarui Electrical Appliance Co., Ltd.\n册地址：宁波市鄞州区姜山镇科技园区\n首次公开发行股票并在创业板上市\n招股意向书\n发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、\n误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连\n带的法律责任。\n发行人及全体董事、监事、高级管理人员、发行人的控股股东、实际控制人\n以及保荐人、承销的证券公司承诺因发行人招股意向书及其他信息披露资料有虚\n假记载、误导性陈述或者重大遗漏，致使投资者在证券发行和交易中遭受损失的，\n将依法赔偿投资者损失。\n\n\n注意：你的回答不需要分段。你的回答格式应该按照下面的内容，请注意---output 等标记都必须输出，这是我用来提取答案的标记。\n除了你的回答之外不要输出多余的文字。\n\n---output\n${"keywords":[你提取的关键词]}\n'
llm(adsasdsd)

'您提供的信息是华瑞电器股份有限公司在创业板市场上市的招股意向书的部分内容。这份声明主要提醒投资者注意创业板市场的投资风险，包括公司业绩不稳定、经营风险高和退市风险大等特点。\n\n此外，发行人及全体董事、监事、高级管理人员承诺招股意向书不存在虚假记载、误导性陈述或重大遗漏，并对其真实性、准确性、完整性、及时性承担个别和连带的法律责任。他们还承诺，如果因为招股意向书或其他信息披露资料有虚假记载、误导性陈述或者重大遗漏，导致投资者在证券发行和交易中遭受损失，将依法赔偿投资者损失。\n\n这是一份标准的招股意向书中的免责声明和责任声明，目的是保护投资者的利益，同时也让投资者了解投资的风险。在进行任何投资决策之前，建议投资者仔细阅读并理解这些声明以及招股意向书中的其他信息。'